In [65]:
import warnings
#from tqdm import tqdm, trange
import pandas as pd 
import streamlit as st 
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
import plotly_express  as px
import plotly.graph_objects as go
import random
import pickle
import requests
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [2]:
def get_stock_data(tickerI):
    tickerL = tickerI
    
    
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{tickerL}?period=annual&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    annual_is_df = pd.DataFrame(response.json())
    if annual_is_df.shape == (0, 0):
        annual_is_df =  pd.DataFrame((float('Nan'), float('Nan')))
        last_date_annual = float('Nan')
    else:
        last_date_annual = annual_is_df.loc[0,'date']
        annual_is_df = annual_is_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        annual_is_df = annual_is_df.add_suffix(f'(FY)')
        
    
    
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{tickerL}?period=quarterly&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    quarterly_is_df = pd.DataFrame(response.json())
    if quarterly_is_df.shape == (0, 0):
        quarterly_is_df =  pd.DataFrame((float('Nan'), float('Nan')))
        last_date_quarterly = float('Nan')
    else:
        last_date_quarterly = quarterly_is_df.loc[0,'date']
        quarterly_is_df = quarterly_is_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        quarterly_is_df = quarterly_is_df.add_suffix(f'(FQ)')
        
    
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{tickerL}?period=annual&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    annual_fp_df = pd.DataFrame(response.json())
    if annual_fp_df.shape == (0, 0):
        annual_fp_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        annual_fp_df = annual_fp_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        annual_fp_df = annual_fp_df.add_suffix(f'(FY)')
    
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{tickerL}?period=quarterly&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    quarterly_fp_df = pd.DataFrame(response.json())
    if quarterly_fp_df.shape == (0, 0):
        quarterly_fp_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        quarterly_fp_df = quarterly_fp_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        quarterly_fp_df = quarterly_fp_df.add_suffix(f'(FQ)')
    
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{tickerL}?period=annual&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    annual_cf_df = pd.DataFrame(response.json())
    if annual_cf_df.shape == (0, 0):
        annual_cf_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        annual_cf_df = annual_cf_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        annual_cf_df = annual_cf_df.add_suffix(f'(FY)')
    
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{tickerL}?period=quarterly&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    quarterly_cf_df = pd.DataFrame(response.json())
    if quarterly_cf_df.shape == (0, 0):
        quarterly_cf_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        quarterly_cf_df = quarterly_cf_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        quarterly_cf_df = quarterly_cf_df.add_suffix(f'(FQ)')
    
    stock_data = pd.concat([annual_is_df, quarterly_is_df, annual_fp_df, quarterly_fp_df, annual_cf_df  ], axis = 1)
    stock_data = pd.concat([stock_data,quarterly_cf_df], axis = 1)
    stock_data = stock_data[stock_data.columns.drop(list(stock_data.filter(regex='calendarYear')))]
    stock_data = stock_data[stock_data.columns.drop(list(stock_data.filter(regex='period')))]
    stock_data = stock_data.loc[:,~stock_data.columns.duplicated()].copy()
    stock_data1 = stock_data.iloc[0:9,]
    
    if quarterly_cf_df.shape[0]<9:
        for year in range(1,quarterly_cf_df.shape[0]):
            stock_data1 = pd.concat([stock_data1, pd.DataFrame(stock_data1.iloc[year,0:198].add_suffix(f'-{year}')).transpose().reset_index(drop=True)], axis=1)
    else:
        for year in range(1,9):
            stock_data1 = pd.concat([stock_data1, pd.DataFrame(stock_data1.iloc[year,0:198].add_suffix(f'-{year}')).transpose().reset_index(drop=True)], axis=1)
    stock_data1 = stock_data1.iloc[0,:]
    stock_data_final = pd.DataFrame(stock_data1).transpose()
    stock_data_final.insert(loc = 0, column ="Ticker", value=f"{tickerL}" )
    stock_data_final.insert(loc = 0, column ="Date(FY)", value=f"{last_date_annual}" )
    stock_data_final.insert(loc = 0, column ="Date(FQ)", value=f"{last_date_quarterly}" )
    
    return stock_data_final

In [3]:
#Download Raw Data
us_stock_database = pd.DataFrame()
stock_list_pd = pd.read_pickle("StockList")
stock_list_pd = stock_list_pd.query("type == 'stock'")
stock_list_pd = stock_list_pd.query("exchangeShortName == 'NYSE' | exchangeShortName == 'NASDAQ'")
stock_list_pd.reset_index(inplace=True, drop=True)
stock_list = stock_list_pd["symbol"].to_list()
for ticker in tqdm(stock_list):
    #tqdm._instances.clear()
    us_stock_database = pd.concat([us_stock_database, get_stock_data(ticker)])
    #print(f'processed {ticker}')
    us_stock_database.reset_index(drop=True)


100%|██████████| 10126/10126 [21:10:38<00:00,  7.53s/it]  


In [84]:
#Create Growth Rate DB
db = pd.read_parquet('NewUSDatabase.parquet')
growth_rates_df = pd.DataFrame(db["Ticker"])
columns = db.columns.to_list()[1:199]
columns
for feature in columns:
    for i in range(1,9):
        if "FY" in feature:
            growth_rates_df[f"{feature} {i} Year Growth Rate"] = round(((db[feature]-db[f"{feature}-{i}"])/db[f"{feature}-{i}"])*100,2)
        else:
            growth_rates_df[f"{feature} {i} Quarter Growth Rate"] = round(((db[feature]-db[f"{feature}-{i}"])/db[f"{feature}-{i}"])*100,2)
growth_rates_df.to_parquet("US_Growth_Rates.parquet")

In [66]:
url = "https://financialmodelingprep.com/api/v3/stock-screener?apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
response = requests.get(url)

In [68]:
stuff = pd.DataFrame(response.json())

In [73]:
stuff = stuff.rename(columns={"symbol":"Ticker"})

In [83]:
stuff

,Ticker,companyName,marketCap,sector,industry,beta,price,lastAnnualDividend,volume,exchange,exchangeShortName,country,isEtf,isFund,isActivelyTrading
0,MSFT,Microsoft Corporation,2956870210400,Technology,Software - Infrastructure,0.877,397.84,3.000000,17207440,NASDAQ Global Select,NASDAQ,US,False,False,True
1,AAPL,Apple Inc.,2671911957000,Technology,Consumer Electronics,1.276,173.03,0.960000,70883853,NASDAQ Global Select,NASDAQ,US,False,False,True
2,NVDA,NVIDIA Corporation,2145425000000,Technology,Semiconductors,1.744,858.17,0.160000,36305611,NASDAQ Global Select,NASDAQ,US,False,False,True
3,GOOG,Alphabet Inc.,2072524507687,Communication Services,Internet Content & Information,1.054,168.46,0.800000,16836892,NASDAQ Global Select,NASDAQ,US,False,False,True
4,GOOGL,Alphabet Inc.,2071279499806,Communication Services,Internet Content & Information,1.054,166.62,0.800000,22900482,NASDAQ Global Select,NASDAQ,US,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,EC,Ecopetrol S.A.,23724335900,Energy,Oil & Gas Integrated,0.756,11.54,3.200000,1671870,New York Stock Exchange,NYSE,CO,False,False,True
996,GIBLX,Guggenheim Total Return Bond Fund Class P,23721095150,Financial Services,Asset Management - Bonds,1.050,22.99,0.764000,0,NASDAQ,NASDAQ,US,False,True,True
997,GIBAX,Guggenheim Total Return Bond Fund Class A,23721095150,Financial Services,Asset Management - Bonds,1.050,22.99,0.764000,0,NASDAQ,NASDAQ,US,False,True,True
998,GIBIX,Guggenheim Total Return Bond Fund,23721095149,Financial Services,Asset Management - Bonds,1.060,23.01,0.900197,0,NASDAQ,NASDAQ,US,False,True,True


In [85]:
growth = pd.read_parquet("US_Growth_Rates.parquet")
growth

,Ticker,revenue(FY) 1 Year Growth Rate,revenue(FY) 2 Year Growth Rate,revenue(FY) 3 Year Growth Rate,revenue(FY) 4 Year Growth Rate,revenue(FY) 5 Year Growth Rate,revenue(FY) 6 Year Growth Rate,revenue(FY) 7 Year Growth Rate,revenue(FY) 8 Year Growth Rate,costOfRevenue(FY) 1 Year Growth Rate,...,capitalExpenditure(FQ) 7 Quarter Growth Rate,capitalExpenditure(FQ) 8 Quarter Growth Rate,freeCashFlow(FQ) 1 Quarter Growth Rate,freeCashFlow(FQ) 2 Quarter Growth Rate,freeCashFlow(FQ) 3 Quarter Growth Rate,freeCashFlow(FQ) 4 Quarter Growth Rate,freeCashFlow(FQ) 5 Quarter Growth Rate,freeCashFlow(FQ) 6 Quarter Growth Rate,freeCashFlow(FQ) 7 Quarter Growth Rate,freeCashFlow(FQ) 8 Quarter Growth Rate
0,NGL-PB,9.40,66.35,14.65,0.07,-48.57,-33.23,-25.95,-48.25,6.67,...,13.11,32.96,70.31,176.48,-74.93,-74.59,-161.28,-238.60,-62.94,950.12
1,NLY-PI,-184.51,-162.03,182.45,-17.09,-700.11,-198.06,-205.54,-324.95,20.97,...,-100.00,-100.00,-77.55,-60.53,-135.89,541.58,-83.33,-10.88,-76.39,-52.24
2,CIO-PA,-0.77,9.18,11.35,14.59,38.32,68.19,147.16,225.32,3.33,...,-inf,-inf,-59.42,-39.69,-39.86,0.89,-59.91,-86.43,-47.30,3.14
3,LEVI,0.17,7.20,38.77,7.22,10.83,26.00,35.72,37.48,1.66,...,52.61,-2.71,5.93,-1111.32,1.85,-178.91,-508.05,-1911.88,1570.95,1609.18
4,ICL,-23.57,10.06,51.78,45.22,37.77,41.28,42.73,41.62,1.84,...,104.67,44.93,-19.50,-21.32,-20.24,-31.81,-58.80,-57.28,-10.37,9.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10121,UNVR,20.34,38.84,23.56,32.93,39.03,42.13,27.76,10.62,21.87,...,10.00,46.02,-674.29,-25.90,10.59,1416.98,-244.52,81.08,157.42,294.76
10122,GOGN-WT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,220.41,93.52,7.73,59.55,-72.27,684.10,724.01,NaN
10123,VHNAW,35.76,1064.15,NaN,NaN,NaN,NaN,NaN,NaN,12.64,...,NaN,NaN,-108.86,-119.58,-115.26,-119.16,NaN,NaN,NaN,NaN
10124,XPAXW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
both = pd.merge(growth, stuff,on="Ticker")

In [80]:
growth

,Ticker,revenue(FY) 1 Year Growth Rate,revenue(FY) 2 Year Growth Rate,revenue(FY) 3 Year Growth Rate,revenue(FY) 4 Year Growth Rate,revenue(FY) 5 Year Growth Rate,revenue(FY) 6 Year Growth Rate,revenue(FY) 7 Year Growth Rate,revenue(FY) 8 Year Growth Rate,costOfRevenue(FY) 1 Year Growth Rate,...,capitalExpenditure(FQ) 7 Quarter Growth Rate,capitalExpenditure(FQ) 8 Quarter Growth Rate,freeCashFlow(FQ) 1 Quarter Growth Rate,freeCashFlow(FQ) 2 Quarter Growth Rate,freeCashFlow(FQ) 3 Quarter Growth Rate,freeCashFlow(FQ) 4 Quarter Growth Rate,freeCashFlow(FQ) 5 Quarter Growth Rate,freeCashFlow(FQ) 6 Quarter Growth Rate,freeCashFlow(FQ) 7 Quarter Growth Rate,freeCashFlow(FQ) 8 Quarter Growth Rate
0,NGL-PB,9.398553,66.345241,14.647996,0.066140,-48.573066,-33.230289,-25.951094,-48.250955,6.672198,...,13.113908,32.963737,70.312949,176.478509,-74.925558,-74.593838,-161.284509,-238.598131,-62.939412,950.118017
1,NLY-PI,-184.509839,-162.032030,182.453611,-17.094465,-700.109369,-198.060044,-205.544638,-324.951103,20.969472,...,-100.000000,-100.000000,-77.545062,-60.534366,-135.885768,541.584460,-83.333722,-10.882110,-76.389935,-52.242937
2,CIO-PA,-0.769593,9.177584,11.350410,14.586972,38.315159,68.185788,147.161922,225.321514,3.333383,...,-inf,-inf,-59.420138,-39.687744,-39.856769,0.887960,-59.908671,-86.432999,-47.295177,3.137098
3,LEVI,0.168596,7.201052,38.772571,7.216844,10.825334,25.998414,35.720497,37.479355,1.660432,...,52.613181,-2.705494,5.928854,-1111.320755,1.852732,-178.910563,-508.054509,-1911.882025,1570.953160,1609.183673
4,ICL,-23.570308,10.056559,51.783337,45.217866,37.768785,41.277846,42.726714,41.617645,1.836559,...,104.667934,44.927023,-19.501303,-21.322540,-20.239823,-31.812868,-58.796875,-57.278333,-10.372585,9.356721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10121,UNVR,20.342929,38.842105,23.564376,32.931364,39.032192,42.131860,27.761696,10.617029,21.866906,...,10.000000,46.017699,-674.285714,-25.898618,10.591472,1416.981132,-244.517675,81.081081,157.417289,294.762684
10122,GOGN-WT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,220.406163,93.516931,7.726853,59.546198,-72.272056,684.101474,724.014551,NaN
10123,VHNAW,35.758006,1064.151128,NaN,NaN,NaN,NaN,NaN,NaN,12.643898,...,NaN,NaN,-108.862251,-119.582991,-115.262235,-119.158673,NaN,NaN,NaN,NaN
10124,XPAXW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
both.to_parquet("US_Growth_Rates.parquet")

In [82]:
both.columns.to_list()

['Ticker',
 'revenue(FY) 1 Year Growth Rate',
 'revenue(FY) 2 Year Growth Rate',
 'revenue(FY) 3 Year Growth Rate',
 'revenue(FY) 4 Year Growth Rate',
 'revenue(FY) 5 Year Growth Rate',
 'revenue(FY) 6 Year Growth Rate',
 'revenue(FY) 7 Year Growth Rate',
 'revenue(FY) 8 Year Growth Rate',
 'costOfRevenue(FY) 1 Year Growth Rate',
 'costOfRevenue(FY) 2 Year Growth Rate',
 'costOfRevenue(FY) 3 Year Growth Rate',
 'costOfRevenue(FY) 4 Year Growth Rate',
 'costOfRevenue(FY) 5 Year Growth Rate',
 'costOfRevenue(FY) 6 Year Growth Rate',
 'costOfRevenue(FY) 7 Year Growth Rate',
 'costOfRevenue(FY) 8 Year Growth Rate',
 'grossProfit(FY) 1 Year Growth Rate',
 'grossProfit(FY) 2 Year Growth Rate',
 'grossProfit(FY) 3 Year Growth Rate',
 'grossProfit(FY) 4 Year Growth Rate',
 'grossProfit(FY) 5 Year Growth Rate',
 'grossProfit(FY) 6 Year Growth Rate',
 'grossProfit(FY) 7 Year Growth Rate',
 'grossProfit(FY) 8 Year Growth Rate',
 'grossProfitRatio(FY) 1 Year Growth Rate',
 'grossProfitRatio(FY) 2 